In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
965,2024-06-07,Austrália Nbl1 Norte,07:00,Gold Coast Rollers,Logan Thunder,1.98,1.75,185.5,1.85,1.85,1.5,1.85,1.60,pGxoNsCe,0.505051,0.571429,0.540541,0.540541,0.076479,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,170.52,0.00,0.0,0.0,0.0,0.0,0.087204,0.000000,0.102479,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
966,2024-06-07,Austrália Nbl1 Sul,07:00,Keilor Thunder,Mt Gambier,1.99,1.74,175.5,1.85,1.85,1.5,1.85,1.61,080ffI24,0.502513,0.574713,0.540541,0.540541,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,232.78,215.60,0.0,0.0,0.0,0.0,0.094786,0.000000,0.098096,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
967,2024-06-07,Austrália Nbl1 Sul,07:00,Waverley,Sandringham Sabres,2.39,1.53,179.5,1.85,1.81,-6.5,2.04,2.40,dzf3hdXG,0.418410,0.653595,0.540541,0.552486,0.072005,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,184.00,0.0,0.0,0.0,0.0,0.310261,0.015456,0.114666,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
968,2024-06-07,Austrália Nbl1 Sul,05:00,Dandenong Rangers F,Frankston F,2.19,1.63,144.5,1.87,1.79,-1.5,1.95,1.67,6574rXyg,0.456621,0.613497,0.534759,0.558659,0.070118,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,161.70,0.0,0.0,0.0,0.0,0.207319,0.030912,0.109387,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
969,2024-06-07,Austrália Nbl1 Oeste,09:30,East Perth Eagles,Joondalup Wolves,1.95,1.77,177.5,1.87,1.83,-4.5,1.83,2.39,ziMk5pmH,0.512821,0.564972,0.534759,0.546448,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,206.40,190.08,0.0,0.0,0.0,0.0,0.068430,0.015289,0.187668,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
970,2024-06-07,Eua Wnba,20:30,Washington Mystics F,Indiana Fever F,1.78,1.88,161.5,1.87,1.93,-2.5,2.00,2.30,h4M9is8n,0.561798,0.531915,0.534759,0.518135,0.093713,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,197.38,129.48,0.0,0.0,0.0,0.0,0.038640,0.022330,0.098666,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
971,2024-06-07,Eua Wnba,23:00,Los Angeles Sparks F,Dallas Wings F,2.15,1.71,160.5,1.85,1.93,1.5,1.97,1.55,C6xBkLxa,0.465116,0.584795,0.540541,0.518135,0.049912,0.0,0.0,NaN,NaN,161.208,27.661183,0.171587,NaN,NaN,NaN,0.000,0.000,173.47,141.96,0.0,0.0,0.0,0.0,0.161206,0.029930,0.168741,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
972,2024-06-07,Indonésia Ibl,10:00,Dewa United,Kesatria Bengawan Solo,1.25,3.69,177.5,1.85,1.85,-8.5,1.83,6.50,O6g43gdt,0.800000,0.271003,0.540541,0.540541,0.071003,0.0,0.0,NaN,NaN,116.728,21.970838,0.188223,NaN,NaN,NaN,0.000,0.000,151.90,0.00,0.0,0.0,0.0,0.0,0.698518,0.000000,0.792842,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
973,2024-06-07,Israel Superliga,08:45,Hapoel Jerusalem,Maccabi Tel Aviv,2.28,1.62,168.5,1.87,1.87,2.5,1.93,1.39,ETXPHqUH,0.438596,0.617284,0.534759,0.534759,0.055880,0.0,0.4,NaN,NaN,150.984,15.695660,0.103956,112.002,20.662019,0.184479,116.554,146.376,166.46,124.70,0.0,0.0,0.0,0.0,0.239328,0.000000,0.230023,0.56,0.112,11.428571,0.0,0.0,0.0,1.43,0.286,2.167832,0.0,0.0,0.0
974,2024-06-07,Itália Série A2,16:00,Fortitudo Bologna,Trapani,2.26,1.59,147.5,1.86,1.81,1.5,2.05,1.48,EcPeIyqo,0.442478,0.628931,0.537634,0.552486,0.071409,0.6,0.4,NaN,NaN,114.836,23.465090,0.204336,101.138,15.673398,0.154970,119.604,110.970,114.39,85.68,0.0,0.0,0.0,0.0,0.246110,0.019267,0.228357,2.32,0.464,2.715517,0.0,0.0,0.0,0.63,0.126,4.682540,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,07:00,Austrália Nbl1 Sul,Waverley,Sandringham Sabres,179.5,1.85,1.0,Over
1,09:30,Austrália Nbl1 Oeste,East Perth Eagles,Joondalup Wolves,177.5,1.87,1.0,Over
2,20:30,Eua Wnba,Washington Mystics F,Indiana Fever F,161.5,1.87,1.0,Over
3,10:00,Indonésia Ibl,Dewa United,Kesatria Bengawan Solo,177.5,1.85,1.0,Over
